获取当前日期和时间
---
注意datetime是模块，datetime模块还包含一个datetime类，通过from datetime import datetime导入的才是datetime这个类。如果仅导入import datetime，则必须引用全名datetime.datetime。

datetime.now()返回当前日期和时间，其类型是datetime。

In [1]:
from datetime import datetime

now = datetime.now()
print(now)
print(type(now))

2018-03-20 22:03:53.915946
<class 'datetime.datetime'>


获取指定日期和时间
---
要指定某个日期和时间，我们直接用参数构造一个datetime。

In [3]:
from datetime import datetime

dt = datetime(2018,3,20,10,21,12)    #年，月，日，时，分，秒
print(dt)

2018-03-20 10:21:12


datetime与timestamp转换
---
在计算机中，时间实际上是用数字表示的。我们把1970年1月1日 00:00:00 UTC+00:00时区的时刻称为epoch time，记为0（1970年以前的时间timestamp为负数），当前时间就是相对于epoch time的秒数，称为timestamp。  
即：  
timestamp = 0 = 1970-1-1 00:00:00 UTC+0:00  
timestamp = 0 = 1970-1-1 08:00:00 UTC+8:00  （北京时间）

可见timestamp的值与时区毫无关系，因为timestamp一旦确定，其UTC时间就确定了，转换到任意时区的时间也是完全确定的，这就是为什么计算机存储的当前时间是以timestamp表示的，因为全球各地的计算机在任意时刻的timestamp都是完全相同的（假定时间已校准）。

In [11]:
from datetime import datetime

t = 1521512472.0
dt = datetime(2018,3,20,10,21,12)

print(datetime.fromtimestamp(t))    #timestamp转换为datetime
print(datetime.timestamp(dt))    # datetime转换为timestamp
print(dt.timestamp())    # datetime转换为timestamp

2018-03-20 10:21:12
1521512472.0
1521512472.0


timestamp是一个浮点数，小数位表示**毫秒数**，某些编程语言（如Java和JavaScript）的timestamp使用整数表示毫秒数，这种情况下只需要把timestamp除以**1000**就得到Python的浮点表示方法。

timestamp没有时区的概念，而datetime是有时区的。上述转换是在timestamp和本地时间做转换。

本地时间是指当前操作系统设定的时区。timestamp也可以直接被转换到UTC标准时区的时间：

In [12]:
from datetime import datetime

t = 1521512472.0
print(datetime.fromtimestamp(t))    # 本地时间
print(datetime.utcfromtimestamp(t))    # UTC时间

2018-03-20 10:21:12
2018-03-20 02:21:12


str转换为datetime
---
datetime.strptime()  
注意转换后的datetime是没有时区信息的。

In [16]:
from datetime import datetime

cday = datetime.strptime('2018-03-20 10:21:12', '%Y-%m-%d %H:%M:%S')
print(cday)

2018-03-20 10:21:12


datetime转换为str
---
strftime()

In [17]:
from datetime import datetime

now = datetime.now()
print(now)
print(now.strftime('%a, %b %d %H:%M'))

2018-03-20 23:18:05.029746
Tue, Mar 20 23:18


datetime加减
---
加减可以直接用+和-运算符，不过需要导入timedelta这个类。

In [19]:
from datetime import datetime, timedelta

now = datetime.now()
print(now)

print(now + timedelta(hours = 10))
print(now - timedelta(days = 1))
print(now + timedelta(days = 2, hours = 12))

2018-03-20 23:21:36.106831
2018-03-21 09:21:36.106831
2018-03-19 23:21:36.106831
2018-03-23 11:21:36.106831


本地时间转换为UTC时间
---
本地时间是指系统设定时区的时间，例如北京时间是UTC+8:00时区的时间，而UTC时间指UTC+0:00时区的时间。

一个datetime类型有一个时区属性tzinfo，但是默认为None，所以无法区分这个datetime到底是哪个时区，除非强行给datetime设置一个时区。

In [24]:
from datetime import datetime, timedelta, timezone

tz_utc_8 = timezone(timedelta(hours=8))    # 创建时区UTC+8:00
now = datetime.now()
dt = now.replace(tzinfo=tz_utc_8)    # 强制设置为UTC+8:00

print(now)
print(dt)


2018-03-20 23:28:24.557781
2018-03-20 23:28:24.557781+08:00


时区转换
---
可以先通过utcnow()拿到当前的UTC时间，再转换为任意时区的时间。

时区转换的关键在于，拿到一个datetime时，要获知其正确的时区，然后强制设置时区，作为基准时间。

利用带时区的datetime，通过astimezone()方法，可以转换到任意时区。

注：不是必须从UTC+0:00时区转换到其他时区，任何带时区的datetime都可以正确转换，例如上述bj_dt到tokyo_dt2的转换。


In [30]:
utc_dt = datetime.utcnow().replace(tzinfo=timezone.utc)
print(utc_dt)

bj_dt = utc_dt.astimezone(timezone(timedelta(hours=8)))
print(bj_dt)

tokyo_dt = utc_dt.astimezone(timezone(timedelta(hours=9)))
print(tokyo_dt)

tokyo_dt2 = bj_dt.astimezone(timezone(timedelta(hours=9)))
print(tokyo_dt2)

2018-03-20 15:34:45.248935+00:00
2018-03-20 23:34:45.248935+08:00
2018-03-21 00:34:45.248935+09:00
2018-03-21 00:34:45.248935+09:00


小结
---
datetime表示的时间需要时区信息才能确定一个特定的时间，否则只能视为本地时间。

如果要存储datetime，最佳方法是将其转换为timestamp再存储，因为timestamp的值与时区完全无关。

In [46]:
# 练习：假设你获取了用户输入的日期和时间如2015-1-21 9:01:30，以及一个时区信息如UTC+5:00，均是str，请编写一个函数将其转换为timestamp：
def to_timestamp(dt_str, tz_str):
    tz = int(tz_str.split(':')[0].replace('UTC',''))
    # tz = int(re.match(r'UTC([+-]\d+):\d+', tz_str).group(1)) 
    return datetime.strptime(dt_str, '%Y-%m-%d %H:%M:%S').replace(tzinfo=timezone(timedelta(hours=tz))).timestamp()

# 测试:
t1 = to_timestamp('2015-6-1 08:10:30', 'UTC+7:00')
assert t1 == 1433121030.0, t1

t2 = to_timestamp('2015-5-31 16:10:30', 'UTC-09:00')
assert t2 == 1433121030.0, t2

print('ok')

ok
